In [ ]:
import os
import pandas as pd
from pathlib import Path 
import re 
from lingua import Language, LanguageDetectorBuilder

In [ ]:
directory = "data"
output_file = 'raw_merged_22_24.txt'
content_list = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            content_list.append(content)

merged_content = '\n\n'.join(content_list)

with open(output_file, 'w', encoding='utf-8') as output:
    output.write(merged_content)

In [ ]:
def read_file_as_string(file_path):
    text = Path(file_path).read_text(encoding='utf-8')
    return text

filepath = 'raw_merged_22_24.txt'
text = read_file_as_string(filepath)

In [ ]:
def split_text_by_identifier_and_content(text):
    # Updated regex pattern to capture identifier + content after ViewReport
    pattern = r'(\S+\s*ID:[\S]+\s+\w{3}\s+\d{2}\s+\w{3}\s+\d{4}\s+\d{2}:\d{2}:\d{2}.*?)\s+ViewReport(.*?)(?=\s+\S+\s*ID:|\Z)'

    # Find all matches based on the pattern
    sections = re.findall(pattern, text, flags=re.DOTALL)

    # Return sections as tuples of (identifier, content)
    result = [(section[0].strip(), section[1].strip()) for section in sections]

    return result


In [ ]:
df = pd.DataFrame(split_text_by_identifier_and_content(text), columns = ['Identifier', 'Text']) 

In [ ]:
def extract_and_process_replies(df):
    df['id'] = df['Identifier'].apply(lambda text: re.findall(r'ID:([^\s]+)', text)[0] if re.findall(r'ID:([^\s]+)', text) else None)
    df['Date'] = df['Identifier'].apply(lambda text: re.findall(r'ID:\S+\s+(\S+\s+\d{2}\s+\S+\s+\d{4}\s+\d{2}:\d{2}:\d{2})', text)[0] if re.findall(r'ID:\S+\s+(\S+\s+\d{2}\s+\S+\s+\d{4}\s+\d{2}:\d{2}:\d{2})', text) else None)
    df['Thread No'] = df['Identifier'].apply(lambda text: re.findall(r'No\.(\d+)', text)[0] if re.findall(r'No\.(\d+)', text) else None)
    df['Quoted By'] = df['Text'].apply(lambda text: re.findall(r'quoted by:\s*>>\d+', text, flags=re.IGNORECASE))
    df['Reply To'] = df['Text'].apply(lambda text: re.findall(r'>>\d+', text))
    
    df['Text'] = df['Text'].apply(lambda text: re.sub(r'quoted by:\s*>>\d+\s*', '', text, flags=re.IGNORECASE).strip())
    df['Text'] = df['Text'].apply(lambda text: re.sub(r'>>\d+\s*', '', text).strip())
    df['Text'] = df['Text'].apply(lambda text: re.sub(r'No\.\d+\s*', '', text).strip())
    
    return df


In [ ]:
df = extract_and_process_replies(df)

In [ ]:
# strip website links from the text
# it means 'image of god' in latin 
sitepattern = r'(?:https?://|www\.)\S+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6}(?:/[^\s]*)?'
df['Text'] = df['Text'].apply(lambda text: re.sub(sitepattern, '', text).strip())
# strip 'imago dei' comments from the text
df = df[~df['Text'].str.contains('imago', case=False, na=False)]
df = df[~df['Text'].str.contains('amplissimus', case=False, na=False)]
# strip Post Reply
postpattern = r'Post\nReply'
df['Text'] = df['Text'].apply(lambda text: re.sub(postpattern, '', text).strip())

In [ ]:
# trying to get rid of this pattern for the millionth time 
metapattern = r'.{5}(sameocrgoogleiqdbsaucenaotrace).*'
df['Text'] = df['Text'].apply(lambda text: re.sub(metapattern, '', text ).strip())
# trying to get rid of this pattern for the millionth time 
metapattern2 = r'.{5}(samegoogleiqdbsaucenaotrace).*'
df['Text'] = df['Text'].apply(lambda text: re.sub(metapattern2, '', text ).strip())

In [ ]:
# LATIN EXTERMINATION!!! 
# lingua-py (https://github.com/pemistahl/lingua-py)
languages = [Language.LATIN, Language.ENGLISH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

def latin_exterminator(s):
    confidence_value = detector.compute_language_confidence(s, Language.LATIN)
    cv = float(f"{confidence_value:.2f}") 
    if cv >= 0.5:
        return None
    else: 
        return s

#use the latin exterminator
df['Text'] = df['Text'].apply(latin_exterminator)
df = df[df['Text'].notnull()]

# drop duplicates by anon-id (this only refers to the post, not the account)
df = df.drop_duplicates(subset = 'id', keep = 'last')

In [ ]:
df.to_csv("nov12_dataset_full.csv")